<a href="https://colab.research.google.com/github/dashakudjaeva/NLP_facult/blob/HW2/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорты

In [1]:
!pip install transformers accelerate datasets evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


In [2]:
import os
import torch
import pandas
import numpy as np
import evaluate
import math
from datasets.utils.tf_utils import dataset_to_tf
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

Загрузка модели

In [4]:
model_name = 'ktrapeznikov/gpt2-medium-topic-news'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

(…)-news/resolve/main/tokenizer_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

(…)dium-topic-news/resolve/main/config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

(…)edium-topic-news/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)edium-topic-news/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)ews/resolve/main/special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

In [5]:
def generate(input_text):
  input_ids = tokenizer.encode(input_text, return_tensors="pt")

  model.eval()
  with torch.no_grad():
      out = model.generate(
          input_ids,
          do_sample=True,
          num_beams=3,
          temperature=1.5,
          top_p=2.2,
          max_length=100,
          repetition_penalty=1.2,
    )
  generated_text = list(map(tokenizer.decode, out))[0]
  print(generated_text)

In [6]:
generate("news about art")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


news about art" that he was going to make it into a book. But then the internet happened and I realised it should be my own book. So I thought, 'well, if I can make it into a book, so can I'. That's when I decided I wanted to write my own autobiography." 		          "I'm not going to lie, I'm a perfectionist - but I'm a perfectionist for what I love.


Загрузка и подготовка датасета

In [7]:
dataset = load_dataset("multi_news")
dataset

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["summary"])

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/44972 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1215 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50))
small_train_dataset

Dataset({
    features: ['document', 'summary', 'input_ids', 'attention_mask'],
    num_rows: 500
})

Вычисление perplexity для загруженной модели

In [10]:
ev_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

In [11]:
ev_dataset1 = tokenized_datasets["test"].shuffle(seed=42).select(range(2))
ev_data=ev_dataset1['summary']
strr_ev=''.join(ev_data)
strr_ev

'– Colorado\'s Animas River isn\'t always bright orange, but that\'s how it looks today after the EPA accidentally spilled a million gallons of mine waste into a tributary. Officials in San Juan County say state officials and the EPA were actually trying to access contaminated water at Gold King Mine in southwest Colorado on Wednesday when they "unexpectedly triggered a large release of mine waste water into the upper portions of Cement Creek," report Time and the Durango Herald. The acidic water—described by one witness as "the filthiest yellow mustard water you\'ve ever seen"—contains sediment and metals that could irritate the skin, reports the AP. There\'s no risk to drinking water, but residents of communities along the river have been urged to cut back on water usage until the plume moves on. Yesterday the plume reached Durango, Colo., which had stopped pumping water out of the river on Wednesday. "It\'s really, really ugly," says a La Plata County official. "Any kind of recreati

In [12]:
def calculatePerplexity(sentence,model,tokenizer):

  input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)
  input_ids = input_ids.to('cpu')
  with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
  loss, logits = outputs[:2]
  return math.exp(loss)

In [13]:
model.eval()
round(calculatePerplexity(strr_ev,model,tokenizer),2)

21.02

Дообучение модели

In [14]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [15]:
training_args = TrainingArguments(
    output_dir="./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=16,
    warmup_steps=9,
    gradient_accumulation_steps=5
)

In [16]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=small_train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None),
)

In [18]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=300, training_loss=3.0700162760416667, metrics={'train_runtime': 411.0336, 'train_samples_per_second': 3.649, 'train_steps_per_second': 0.73, 'total_flos': 754995573411840.0, 'train_loss': 3.0700162760416667, 'epoch': 3.0})

In [19]:
model.save_pretrained('model_newsss')

In [20]:
def generate1(input_text):
  input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda:0')

  model.eval()
  with torch.no_grad():
      out = model.generate(
          input_ids,
          do_sample=True,
          num_beams=3,
          temperature=1.5,
          top_p=2.2,
          max_length=100,
          repetition_penalty=1.2,
          pad_token_id=50256)
  generated_text = list(map(tokenizer.decode, out))[0]
  print(generated_text)

In [27]:
generate1("criminal news")

criminal news" - but it didn't seem to get the attention it deserved, according to a post-mortem report released today.The investigation by San Diego County coroner's office found the 49-year-old died from blunt force trauma to the head, which was consistent with being struck by a blunt object, reports CBS San Diego. The autopsy also found that the victim had multiple abrasions on her body. The cause of death is still under investigation, but investigators say they believe it was an accident


In [22]:
generate1("news about politics")

news about politics and the world. There's also plenty of political banter, and a chance to talk to a number of top politicians. The event is being hosted by the New York Times, which has invited more than 100 people to share their views, including the presidential hopefuls. You may be surprised to learn that no one has ever won a debate in the history of the Times, but it's a chance to give a voice to the people, even if you don't agree with everything they


In [23]:
generate1("news about art")

news about art and design, the arts and the arts in general, and the importance of the arts in society. We have been invited to join in an art walk along the Thames this year. It is a celebration of the arts and the way that they are used in everyday life and the way they are seen by people around the world. As part of the walk, we'll be looking at the history of art, and what it means to people today, and what we can learn from it.


 Оценка качества (вычисление perplexity для дообученной модели)



In [30]:
eval_results=trainer.evaluate(ev_dataset)
print(f">>>Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>>Perplexity: 18.50


Значение perplexity после дообучения модели меньше, чем до (18,50<21,02), а значит, дообучение прошло успешно